In [116]:
import torch
import torch.nn.functional as F

In [117]:
torch.manual_seed(1337)

In [118]:
### Example of Self-Attention: A Decoder Block
batch_size = 4
context_length = 8
n_emb = 32
head_size = 16


x = torch.randn(batch_size, context_length, n_emb)
key = torch.nn.Linear(n_emb, head_size ,bias = False)
query = torch.nn.Linear(n_emb, head_size, bias = False)
value = torch.nn.Linear(n_emb, head_size, bias = False)
k = key(x)  ## Batch_size x context_length x head_size
q = query(x) ## Batch_size x context_length x head_size
weights = q@k.transpose(-2, -1) ## Perform Matmul keeping the Batch
weights = weights * head_size**-0.5 ## Normalize


In [119]:
tril = torch.tril(torch.ones(context_length, context_length)) ## Initialize Attention Mat (Lower Triangular)
# weights = torch.zeros((context_length,context_length))
weights = weights.masked_fill(tril==0,float('-inf'))  #Delete this to change to encoder block
weights = F.softmax(weights,dim = -1)
weights[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3966, 0.6034, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3069, 0.2892, 0.4039, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3233, 0.2175, 0.2443, 0.2149, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1479, 0.2034, 0.1663, 0.1455, 0.3369, 0.0000, 0.0000, 0.0000],
        [0.1259, 0.2490, 0.1324, 0.1062, 0.3141, 0.0724, 0.0000, 0.0000],
        [0.1598, 0.1990, 0.1140, 0.1125, 0.1418, 0.1669, 0.1061, 0.0000],
        [0.0845, 0.1197, 0.1078, 0.1537, 0.1086, 0.1146, 0.1558, 0.1553]],
       grad_fn=<SelectBackward0>)

In [120]:
v = value(x)
output = weights @ v ##(B,context_length x context_length) @(B x context_length x head_size)

output.shape

torch.Size([4, 8, 16])

In [121]:
print('context length', output[0].shape[0],'n_emb',output[0].shape[1])
output[0]

context length 8 n_emb 16


tensor([[-1.5713e-01,  8.8009e-01,  1.6152e-01, -7.8239e-01, -1.4289e-01,
          7.4676e-01,  1.0068e-01, -5.2395e-01, -8.8726e-01,  1.9068e-01,
          1.7616e-01, -5.9426e-01, -4.8124e-01, -4.8598e-01,  2.8623e-01,
          5.7099e-01],
        [ 4.3974e-01, -1.4227e-01, -1.3157e-01,  2.8895e-03, -1.3222e-01,
          6.6093e-04, -2.7904e-01, -2.2676e-01, -2.8723e-01,  5.7456e-01,
          5.6053e-01, -2.5208e-01,  9.7243e-02,  1.0771e-01,  3.0455e-02,
          1.0727e+00],
        [ 4.3615e-01, -6.6358e-02, -2.9296e-01,  7.4315e-02,  5.4381e-02,
         -7.0388e-02, -6.8985e-02, -8.2153e-02, -2.9377e-01, -5.8952e-02,
          3.5887e-01, -2.3087e-03, -1.8212e-01, -3.6142e-02, -6.7189e-02,
          1.1412e+00],
        [ 4.2068e-01, -1.0619e-01, -2.9984e-01,  5.2820e-02,  2.0077e-01,
         -1.6048e-01, -3.5710e-02, -8.3110e-02, -1.7919e-01,  7.7992e-02,
          1.2719e-01,  2.2611e-02, -5.1811e-02,  7.4466e-02,  1.8131e-01,
          8.4463e-01],
        [ 3.9499e-01